In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
rnn = nn.RNN(10, 20)
print(rnn.num_layers)
input = autograd.Variable(torch.randn(5, 3, 10))
hidden = autograd.Variable(torch.randn(1, 3, 20))
output, hn = rnn(input, hidden)
# print (output.type)
print (hn.type)

In [ ]:
lstm = nn.LSTM(3, 3)
print (lstm.num_layers)
inputs = [autograd.Variable(torch.randn(1, 3)) for _ in range(5)]
# print (inputs)
hidden = (autograd.Variable(torch.Tensor(1, 1, 3)),
         autograd.Variable(torch.Tensor(1, 1, 3)))
# print (hidden)

for i in inputs:
    output, h = lstm(i, hidden)
    # print (h)

inputs1 = torch.cat(inputs)
print (inputs1.type)
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print (inputs.type)

In [ ]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

def prepare_sequence(seq, words_to_index):
    indices = [words_to_index[w] for w in seq]
    return autograd.Variable(torch.LongTensor(indices))

word_to_index = {}
for sent, _ in training_data:
    for word in sent:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
print (word_to_index)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM=6
HIDDEN_DIM=6

In [ ]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, target_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.Tensor(1, 1, self.hidden_dim)),
                autograd.Variable(torch.Tensor(1, 1, self.hidden_dim)))    
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print ("Embeddings type is ", embeds.type)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        linear_out = self.hidden2tag(lstm_out.view(len(sentence), -1))
        return F.log_softmax(linear_out)

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_index), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

sentence = prepare_sequence(training_data[0][0], word_to_index)
# print (sentence)
tags_score = model(sentence)
print (tags_score)

In [ ]:
for epoch in range(2):
    for sent, tags in training_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        # print (model.hidden)
        
        sentence_in = prepare_sequence(sent, word_to_index)
        print (sentence_in)
        tags_in = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        print (tag_scores)
        
        loss = loss_function(tag_scores, tags_in)
        loss.backward()
        optimizer.step()
        
sentence = prepare_sequence(training_data[0][0], word_to_index)
# print (sentence)
tags_score = model(sentence)
print (tags_score)        